# Show4DSTEM Batch CLI + Adaptive Export Demo

Run the non-interactive CLI for single/frame/adaptive exports and inspect JSONL/manifest outputs.

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    %env ANYWIDGET_HMR=1
except Exception:
    pass

In [ ]:
import json
import math
import pathlib

import numpy as np
import torch
from quantem.widget import Show4DSTEM


def make_crystal_4dstem(
    scan_shape=(40, 40),
    det_shape=(72, 72),
    n_frames=1,
    seed=7,
    device=None,
):
    if device is None:
        device = torch.device(
            "mps"
            if torch.backends.mps.is_available()
            else "cuda"
            if torch.cuda.is_available()
            else "cpu"
        )
    torch.manual_seed(seed)

    sy, sx = scan_shape
    ky, kx = det_shape
    fy = 1.0 if n_frames == 1 else float(n_frames)

    y = torch.linspace(-1.0, 1.0, sy, device=device)
    x = torch.linspace(-1.0, 1.0, sx, device=device)
    Y, X = torch.meshgrid(y, x, indexing="ij")

    frame_axis = torch.linspace(0.0, 1.0, n_frames, device=device)
    phase_t = 2.0 * math.pi * frame_axis[:, None, None]

    # Synthetic strain/defect field with crystal-like periodicity.
    lattice = 0.35 * torch.sin(2.0 * math.pi * (7.0 * X + 0.35 * Y))
    lattice += 0.30 * torch.sin(2.0 * math.pi * (7.0 * Y - 0.25 * X))
    defect = 0.55 * torch.exp(-((X + 0.20) ** 2 + (Y - 0.25) ** 2) / 0.05)
    defect += 0.35 * torch.exp(-((X - 0.35) ** 2 + (Y + 0.10) ** 2) / 0.03)
    structural = lattice + defect

    structural_t = structural[None, :, :] + 0.08 * torch.sin(phase_t + 5.0 * X[None, :, :])

    ky_axis = torch.linspace(0.0, float(ky - 1), ky, device=device)
    kx_axis = torch.linspace(0.0, float(kx - 1), kx, device=device)
    KY, KX = torch.meshgrid(ky_axis, kx_axis, indexing="ij")
    KY = KY[None, None, None, :, :]
    KX = KX[None, None, None, :, :]

    cy = (ky - 1) / 2.0
    cx = (kx - 1) / 2.0

    # Shift Bragg spots according to local strain/field.
    shift_y = 1.4 * structural_t[:, :, :, None, None]
    shift_x = -1.1 * structural_t[:, :, :, None, None]

    frame_mod = 1.0 + 0.20 * torch.sin(phase_t)[:, :, :, None, None]

    dp = torch.zeros((n_frames, sy, sx, ky, kx), device=device)
    sigma_direct = 2.8
    direct_amp = 180.0 * frame_mod
    dp += direct_amp * torch.exp(-((KY - cy) ** 2 + (KX - cx) ** 2) / (2.0 * sigma_direct**2))

    spot_sigma = 2.1
    base_spots = [
        (-16.0, 0.0, 75.0),
        (16.0, 0.0, 75.0),
        (0.0, -16.0, 72.0),
        (0.0, 16.0, 72.0),
        (-11.0, -11.0, 58.0),
        (11.0, 11.0, 58.0),
        (-11.0, 11.0, 54.0),
        (11.0, -11.0, 54.0),
    ]

    amp_field = 1.0 + 0.45 * torch.relu(structural_t)[:, :, :, None, None]
    for dy, dx, amp in base_spots:
        cy_t = cy + dy + shift_y
        cx_t = cx + dx + shift_x
        dp += (amp * amp_field) * torch.exp(
            -((KY - cy_t) ** 2 + (KX - cx_t) ** 2) / (2.0 * spot_sigma**2)
        )

    ring_r = torch.sqrt((KY - cy) ** 2 + (KX - cx) ** 2)
    dp += 12.0 * torch.exp(-((ring_r - 23.0) ** 2) / (2.0 * 7.0**2))

    if device.type == "mps":
        # MPS currently lacks aten::poisson; sample on CPU then move back.
        shot = torch.poisson(torch.clamp(dp.cpu(), min=0.0)).to(device)
    else:
        shot = torch.poisson(torch.clamp(dp, min=0.0))
    dp = shot + 0.6 * torch.randn_like(shot)
    dp = torch.clamp(dp, min=0.0).to(torch.float32)

    arr = dp.cpu().numpy()
    if n_frames == 1:
        return arr[0]
    return arr


DEVICE = torch.device(
    "mps"
    if torch.backends.mps.is_available()
    else "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using device: {DEVICE}")

import subprocess
import sys


In [ ]:
OUT = pathlib.Path('notebooks/show4dstem/paper_cli')
OUT.mkdir(parents=True, exist_ok=True)
input_path = OUT / 'synthetic_4dstem.npy'

data = make_crystal_4dstem(scan_shape=(36, 36), det_shape=(72, 72), n_frames=3, seed=33, device=DEVICE)
np.save(input_path, data)

preview = Show4DSTEM(data, pixel_size=0.80, k_pixel_size=0.53)
preview.auto_detect_center()
preview.roi_circle()
preview.roi_radius = 12
preview

In [ ]:
batch_dir = OUT / 'batch_outputs'
cmd = [
    sys.executable,
    '-m',
    'quantem.widget.show4dstem_batch',
    '--input', str(input_path),
    '--output-dir', str(batch_dir),
    '--mode', 'adaptive',
    '--adaptive-target-fraction', '0.30',
    '--adaptive-coarse-step', '4',
    '--adaptive-min-spacing', '2',
    '--view', 'all',
    '--format', 'png',
    '--manifest-name', 'adaptive_manifest.json',
    '--batch-manifest', 'batch_manifest.jsonl',
    '--report-name', 'adaptive_report.json',
]

res = subprocess.run(cmd, check=True, capture_output=True, text=True)
print(res.stdout)

In [ ]:
manifest_rows = [json.loads(line) for line in (batch_dir / 'batch_manifest.jsonl').read_text().splitlines() if line.strip()]
manifest_rows

In [ ]:
sequence_manifest = json.loads((batch_dir / 'adaptive_manifest.json').read_text())
report = json.loads((batch_dir / 'adaptive_report.json').read_text())

summary = {
    'metadata_version': sequence_manifest['metadata_version'],
    'widget_version': sequence_manifest['widget_version'],
    'n_exports': sequence_manifest['n_exports'],
    'include_overlays': sequence_manifest['include_overlays'],
    'include_scalebar': sequence_manifest['include_scalebar'],
    'report_n_exports': report['n_exports'],
}
summary

In [ ]:
sorted(str(p.relative_to(OUT)) for p in OUT.rglob('*') if p.is_file())